In [1]:
#content based
import pandas as pd
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data=pd.read_csv(r"recommend.csv")

In [3]:
data

,movie_id,title,genres,actors,director,description
0,1,The Shawshank Redemption,Drama,Morgan,Frank D,Two imprisoned men bond over a number of years...
1,2,The Godfather,Crime,Marlon,Francis F,The aging patriarch of an organized crime dyna...
2,3,The Dark Knight,Action,Christian,Christopher Nolan,When the menace known as the Joker wreaks havo...
3,4,Inception,Action,Leonardo,Christopher Nolan,A thief who steals corporate secrets through t...
4,5,Forrest Gump,Drama,Tom,Robert Zemeckis,"The presidencies of Kennedy and Johnson, the e..."


In [4]:
#tfidf calculation
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['director'])
#director column is used to calculate tf-idf

In [5]:
data['director']

0              Frank D
1            Francis F
2    Christopher Nolan
3    Christopher Nolan
4      Robert Zemeckis
Name: director, dtype: object

In [6]:
#tfidf.vocabulary_

In [7]:
data

,movie_id,title,genres,actors,director,description
0,1,The Shawshank Redemption,Drama,Morgan,Frank D,Two imprisoned men bond over a number of years...
1,2,The Godfather,Crime,Marlon,Francis F,The aging patriarch of an organized crime dyna...
2,3,The Dark Knight,Action,Christian,Christopher Nolan,When the menace known as the Joker wreaks havo...
3,4,Inception,Action,Leonardo,Christopher Nolan,A thief who steals corporate secrets through t...
4,5,Forrest Gump,Drama,Tom,Robert Zemeckis,"The presidencies of Kennedy and Johnson, the e..."


In [8]:
import numpy as np

In [9]:
vect =  tfidf.transform(list(data['director'])).toarray() #
s1 = np.array(vect[2])
s2 = np.array(vect[3])

In [10]:
s1

array([0.70710678, 0.        , 0.        , 0.70710678, 0.        ,
       0.        ])

In [11]:
#s2 = s2.reshape(-1 , 1)

In [12]:
#s1 = s1.reshape(-1 , 1)

In [13]:
# Compute cosine similarity matrix
def cosine(idx,a):
    import math
    s1 = np.array(vect[idx])
    s2 = np.array(vect[a])
    dot=np.dot(s1,s2)
    magnitude1=math.sqrt(sum(pow(element, 2) for element in s1))
    magnitude2=math.sqrt(sum(pow(element, 2) for element in s2))
    cosine_sim=dot/(magnitude1*magnitude2)
    cosine_sim=int(cosine_sim)
    return cosine_sim

In [14]:
# Get indices and titles of all movies
indices = pd.Series(data.index, index=data['title']).drop_duplicates()

In [15]:
indices[:1]

title
The Shawshank Redemption    0
dtype: int64

In [16]:
def get_recommendations(title,indices=indices):
    # Get index of the movie
    idx = indices[title]
    print(idx)

    # Get similarity scores of all movies
    sim_scores=[]
    indices_taken={}
    for i in indices:
        if i!=idx: #idx is the index of the movie we want it to recommend
            print("i is",i)
            print(idx)
            print(i)
            value=cosine(idx,i)
            sim_scores.append(value)
            print("sim scores is",sim_scores)
            indices_taken[i]=value
    print(sim_scores)
    print("indices_taken is",indices_taken)

    # Sort movies by similarity score
    dict={}
    sim_scores.sort(reverse=True)
    for w in sorted(indices_taken, key=indices_taken.get, reverse=True):
        print(w, indices_taken[w])
        dict[w]=indices_taken[w]
    
    # Get indices of top 1 most similar movies
    sim_scores = sim_scores[0:1]
    print("Top 2 sim_scores are",sim_scores)
    print("Top 2 movies are",list(dict.items())[:1])
    for i in list(dict.items())[:1]:
        print(i[0])
        val=i[0]
        #val is index of most similar item to be recommended
        print("Recommendation:",indices[val:val+1])

In [17]:
value=get_recommendations('The Dark Knight',indices)

2
i is 0
2
0
sim scores is [0]
i is 1
2
1
sim scores is [0, 0]
i is 3
2
3
sim scores is [0, 0, 1]
i is 4
2
4
sim scores is [0, 0, 1, 0]
[0, 0, 1, 0]
indices_taken is {0: 0, 1: 0, 3: 1, 4: 0}
3 1
0 0
1 0
4 0
Top 2 sim_scores are [1]
Top 2 movies are [(3, 1)]
3
Recommendation: title
Inception    3
dtype: int64


In [18]:
value